In [ ]:
import cv2
import numpy as np
from gtts import gTTS
import os
from playsound import playsound
def nothing(x):
    pass


distance = 0
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)
font = cv2.FONT_HERSHEY_COMPLEX
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
Kernal = np.ones((3, 3), np.uint8)
points = []
# backSub = cv2.createBackgroundSubtractorKNN()
backSub = cv2.createBackgroundSubtractorMOG2()
while (1):
    ret, frame = cap.read()
    frame = cv2.flip(frame, +1)
    if not ret:
        break
    if cv2.waitKey(int(1000/fps)) == ord('s'):
        break
    frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lb = np.array([30, 85, 100])
    ub = np.array([72, 255, 255])

    # for red ball
    
#     l_b = np.array([0,50,50])
#     u_b = np.array([10, 255, 255])
    
    mask = cv2.inRange(frame2, lb, ub)
    cv2.imshow('Masked Image', mask)
    
    Mask = backSub.apply(frame)
    
    cv2.imshow('backgrd mask', Mask)
    
    f_mask = cv2.bitwise_and(mask,Mask)
    res1 = cv2.bitwise_and(frame , frame , mask = f_mask)
    
    cv2.imshow('res1', res1)
    
    cv2.imshow('f_mask', f_mask)
    
    opening = cv2.morphologyEx(f_mask,cv2.MORPH_OPEN, Kernal)
    cv2.imshow('Opening', opening)

    
    
    res2 = cv2.bitwise_and(res1, res1, mask=opening)
    cv2.imshow('Resuting Image', res2)

    contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_NONE)

    if len(contours) != 0:
        cnt = contours[0]
        area = cv2.contourArea(cnt)
        if(area > 100):
            M = cv2.moments(cnt)
            # print(M)
            Cx = int(M['m10'] / M['m00'])
            Cy = int(M['m01'] / M['m00'])
            distance = 0.0003 * (Cy ** (2)) - (0.3178 * Cy) + 103.5
            points.append((Cx, Cy))
            
            if (Cx == 640):
                cv2.putText(frame, 'MIDDLE', (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
                mytext = "Middle"
                language = 'en'

                output = gTTS(text=mytext, lang=language, slow=False)
                output.save("middle.mp3")

                os.system("start middle.mp3")
            if (Cx < 640):
                cv2.putText(frame, 'LEFT', (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
                #cv2.putText(frame, 'MIDDLE', (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
                #mytext = "Left"
                #language = 'en'

                #output = gTTS(text=mytext, lang=language, slow=False)
                #output.save("left.mp3")

                #os.system("start left.mp3")
                playsound('Left.mp3')
            if (Cx > 640):
                cv2.putText(frame, 'RIGHT', (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
                #cv2.putText(frame, 'MIDDLE', (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
                #mytext = "right"
                #language = 'en'

                #output = gTTS(text=mytext, lang=language, slow=False)
                #output.save("right.mp3")

                #os.system("start right.mp3")
                playsound('right.mp3')

            # if(len(points)>=2):

            # cv2.line(frame,points[-1],points[-2],(255,0,0),5)

            # S = 'Location of object:' + '(' + str(Cx) + ',' + str(Cy) + ')'
            # cv2.putText(frame, S, (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
            # S = 'Area of contour: ' + str(area)
            # cv2.putText(frame, S, (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
        S = 'Distance Of Object: ' + str(distance)
        cv2.putText(frame, S, (5, 50), font, 2, (0, 0, 255), 2, cv2.LINE_AA)
        
        
        
        cv2.drawContours(frame, cnt, -1, (0, 255, 0), 3)

    for i in range(1, len(points)):
        if points[i - 1] is None or points[i] is None:
            continue

        cv2.line(frame, points[i - 1], points[i], (0, 0, 255), 5)
    if (len(points) == 5):
        points.pop(0)

    ##Lets Detect a red ball
    cv2.imshow('Original Image', frame)

    # if(len(points)>=2):
    # cv2.line(frame,points[-1],points[-2],(255,0,0),5)

#     cv2.imshow('Original Image', frame)

cap.release()
cv2.destroyAllWindows()